# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [5]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,mahina,-17.5065,-149.4890,26.87,74,75,1.03,PF,1702882223
1,1,wailua homesteads,22.0669,-159.3780,20.44,87,75,7.20,US,1702882224
2,2,grand-bassam,5.2118,-3.7388,25.98,85,9,0.93,CI,1702882224
3,3,selkirk,50.1436,-96.8845,-12.96,77,61,1.34,CA,1702882225
4,4,edinburgh of the seven seas,-37.0676,-12.3116,12.44,68,10,1.07,SH,1702882225


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [19]:
%%capture --no-display

# Configure the map plot

humidity_map = city_data_df.hvplot.points(
    'Lng', 'Lat', 
    geo=True, 
    size='Humidity', 
    color='City', 
    scale=0.8,  
    tiles='OSM', 
    frame_width=1000, 
    frame_height=500,
    hover_cols=['City'] 
).opts(
    global_extent=True, 
    title="City Humidity Map",
    tools=['hover']  
)

# Display the map
humidity_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [15]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather = city_data_df[(city_data_df["Max Temp"] >= 10) &
                                (city_data_df["Max Temp"] < 30)
                             ]

# Drop any rows with null values
ideal_weather.dropna()

# Display sample data
ideal_weather

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,mahina,-17.5065,-149.4890,26.87,74,75,1.03,PF,1702882223
1,1,wailua homesteads,22.0669,-159.3780,20.44,87,75,7.20,US,1702882224
2,2,grand-bassam,5.2118,-3.7388,25.98,85,9,0.93,CI,1702882224
4,4,edinburgh of the seven seas,-37.0676,-12.3116,12.44,68,10,1.07,SH,1702882225
7,7,hermanus,-34.4187,19.2345,19.10,71,98,4.55,ZA,1702882227
...,...,...,...,...,...,...,...,...,...,...
579,579,praia da vitoria,38.7333,-27.0667,19.14,88,20,6.36,PT,1702882531
580,580,banda aceh,5.5577,95.3222,27.75,80,90,1.64,ID,1702882531
581,581,kapa'a,22.0752,-159.3190,21.18,82,75,7.20,US,1702882532
582,582,geraldton,-28.7667,114.6000,29.23,45,8,10.29,AU,1702882532


### Step 3: Create a new DataFrame called `hotel_df`.

In [16]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = " "

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
0,mahina,PF,-17.5065,-149.4890,74,
1,wailua homesteads,US,22.0669,-159.3780,87,
2,grand-bassam,CI,5.2118,-3.7388,85,
4,edinburgh of the seven seas,SH,-37.0676,-12.3116,68,
7,hermanus,ZA,-34.4187,19.2345,71,
...,...,...,...,...,...,...
579,praia da vitoria,PT,38.7333,-27.0667,88,
580,banda aceh,ID,5.5577,95.3222,80,
581,kapa'a,US,22.0752,-159.3190,82,
582,geraldton,AU,-28.7667,114.6000,45,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [26]:
# Set parameters to search for a hotel
geoapify_key = "d01fb0f006fe4f77b0eb3740199c7b06"
radius = 10000
categories = ["accommodation.hotel"]
params = {
    "categories": categories,
    "apiKey": geoapify_key,
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    Latitude = row["Lat"]
    Longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{Longitude},{Latitude},{radius}"
    params["bias"] = f"proximity:{Longitude},{Latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params = params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
mahina - nearest hotel: Motu Martin
wailua homesteads - nearest hotel: Hilton Garden Inn Kauai Wailua Bay
grand-bassam - nearest hotel: Hôtel 3G
edinburgh of the seven seas - nearest hotel: No hotel found
hermanus - nearest hotel: Aloe guest house
barclayville - nearest hotel: No hotel found
margaret river - nearest hotel: Margaret River Hotel
mityana - nearest hotel: Kolping Hotel Masindi
bamboo flat - nearest hotel: Hotel Megapode Nest
urangan - nearest hotel: Ramada
santa ines - nearest hotel: Hotel Carajás
hawaiian paradise park - nearest hotel: No hotel found
porto novo - nearest hotel: Hôtel 6500 CFA
waitangi - nearest hotel: Hotel Chathams
hithadhoo - nearest hotel: Pebbles Inn
tzucacab - nearest hotel: No hotel found
kidal - nearest hotel: No hotel found
imbituba - nearest hotel: Silvestre Praia Hotel
adamstown - nearest hotel: No hotel found
touros - nearest hotel: Pousada Atlântico
kavaratti - nearest hotel: No hotel found
saint-joseph - nearest hotel: P

,City,Country,Lat,Lng,Humidity,Hotel Name
0,mahina,PF,-17.5065,-149.4890,74,Motu Martin
1,wailua homesteads,US,22.0669,-159.3780,87,Hilton Garden Inn Kauai Wailua Bay
2,grand-bassam,CI,5.2118,-3.7388,85,Hôtel 3G
4,edinburgh of the seven seas,SH,-37.0676,-12.3116,68,No hotel found
7,hermanus,ZA,-34.4187,19.2345,71,Aloe guest house
...,...,...,...,...,...,...
579,praia da vitoria,PT,38.7333,-27.0667,88,Salles
580,banda aceh,ID,5.5577,95.3222,80,Hotel 61
581,kapa'a,US,22.0752,-159.3190,82,Pono Kai Resort
582,geraldton,AU,-28.7667,114.6000,45,Ocean Centre Hotel


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [30]:
%%capture --no-display

# Configure the map plot
hotels_map = hotel_df.hvplot.points(
    'Lng', 'Lat', 
    geo=True, 
    size='Humidity', 
    color='City', 
    scale=0.8,  
    tiles='OSM', 
    frame_width=1000, 
    frame_height=500,
    hover_cols=["Hotel Name", "Country"] 
).opts(
    global_extent=True, 
    tools=['hover']  
)

# Display the map
hotels_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)